In [256]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML
import subprocess
import time as tme
from matplotlib import cm
from matplotlib.colors import Normalize
norm = Normalize()

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [257]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
csfont = {'fontname':'Times New Roman'}

#Lists
#RLength
Hx=1.0
Hy=10.0
L = 0.1
ReList=[0.1,1.0,2.0,7.0,10.0,25.0,50.0]


In [258]:
def CheckBounds(delta,val):
    global L
    if delta > L/2.0:
        #print('Crossed Boundary')
        delta = delta - L
    elif delta < -L/2.0:
        #print('Crossed Boundary')
        delta = delta + L
    newval = val + delta
    return (delta,newval)

def FixPeriodicBounds(posData):
    #Account for Periodic Conditions
    #Whenever a sphere crosses the boundary
    posData['adXU'] = 0.0
    posData['adYU'] = 0.0
    posData['adXL'] = 0.0
    posData['adYL'] = 0.0
    posData['bdXU'] = 0.0
    posData['bdYU'] = 0.0
    posData['bdXL'] = 0.0
    posData['bdYL'] = 0.0
    posData['aXUn'] = posData['aXU'].copy()
    posData['aYUn'] = posData['aYU'].copy()
    posData['aXLn'] = posData['aXL'].copy()
    posData['aYLn'] = posData['aYL'].copy()
    posData['bXUn'] = posData['bXU'].copy()
    posData['bYUn'] = posData['bYU'].copy()
    posData['bXLn'] = posData['bXL'].copy()
    posData['bYLn'] = posData['bYL'].copy()
    for idx in range(1,len(posData['time'])):
        #Swimmer 1
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'adXU'] = posData.loc[idx,'aXU'] - posData.loc[idx-1,'aXUn']
        posData.loc[idx,'adYU'] = posData.loc[idx,'aYU'] - posData.loc[idx-1,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXU'], posData.loc[idx,'aXUn'] = CheckBounds(posData.loc[idx,'adXU'],posData.loc[idx-1,'aXUn'])
        posData.loc[idx,'adYU'], posData.loc[idx,'aYUn'] = CheckBounds(posData.loc[idx,'adYU'],posData.loc[idx-1,'aYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'adXL'] = posData.loc[idx,'aXL'] - posData.loc[idx,'aXUn']
        posData.loc[idx,'adYL'] = posData.loc[idx,'aYL'] - posData.loc[idx,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXL'], posData.loc[idx,'aXLn'] = CheckBounds(posData.loc[idx,'adXL'],posData.loc[idx,'aXUn'])
        posData.loc[idx,'adYL'], posData.loc[idx,'aYLn'] = CheckBounds(posData.loc[idx,'adYL'],posData.loc[idx,'aYUn'])
        #Swimmer 2
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'bdXU'] = posData.loc[idx,'bXU'] - posData.loc[idx-1,'bXUn']
        posData.loc[idx,'bdYU'] = posData.loc[idx,'bYU'] - posData.loc[idx-1,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXU'], posData.loc[idx,'bXUn'] = CheckBounds(posData.loc[idx,'bdXU'],posData.loc[idx-1,'bXUn'])
        posData.loc[idx,'bdYU'], posData.loc[idx,'bYUn'] = CheckBounds(posData.loc[idx,'bdYU'],posData.loc[idx-1,'bYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'bdXL'] = posData.loc[idx,'bXL'] - posData.loc[idx,'bXUn']
        posData.loc[idx,'bdYL'] = posData.loc[idx,'bYL'] - posData.loc[idx,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXL'], posData.loc[idx,'bXLn'] = CheckBounds(posData.loc[idx,'bdXL'],posData.loc[idx,'bXUn'])
        posData.loc[idx,'bdYL'], posData.loc[idx,'bYLn'] = CheckBounds(posData.loc[idx,'bdYL'],posData.loc[idx,'bYUn'])

    posDict = {'aXU':posData['aXUn'],'aYU':posData['aYUn'],'aXL':posData['aXLn'],'aYL':posData['aYLn'],
                    'bXU':posData['bXUn'],'bYU':posData['bYUn'],
                    'bXL':posData['bXLn'],'bYL':posData['bYLn'],
                    'time':posData['time']}
    posData = pd.DataFrame(data=posDict)
    return posData

def CentralDifference(data,idx,dx):
    return (data[idx+1] - data[idx-1])/(2.0*dx)

def CalcLabAngle(nx,ny):
    theta = np.zeros(len(nx))
    theta = np.arccos(nx)
    theta = np.where(ny < 0.0, -1.0*theta+2.0*np.pi,theta)
    return theta
    

def CalcAngularFrequency(theta):
    global PERIOD
    #Calculate Angular Velocity for swimmers A and B
    #Angles are in range [0,2pi). 
    #Velocities will be [-pi,pi]
    w = np.zeros(len(theta))
    for idx in range(1,len(w)):
        diff1 = theta[idx] - theta[idx-1]
        if(diff1 <= -np.pi):
            diff = diff1 + 2.0*np.pi
        elif(diff1 >= np.pi):
            diff = diff1 - 2.0*np.pi
        else:
            diff = diff1
        w[idx] = diff/PERIOD
    return w

def StoreData(cwd_POSDATA, Re):
    global RADIUSLARGE,RADIUSSMALL,Hx,Hy,PERIOD
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData = pd.read_csv(cwd_POSDATA+'pd.txt',delimiter=' ')
    #Split up individual sphere data by given index
    UAdata = pdData[pdData['idx'] == 6].copy()
    LAdata = pdData[pdData['idx'] == 19].copy()
    UBdata = pdData[pdData['idx'] == 32].copy()
    LBdata = pdData[pdData['idx'] == 45].copy()
    #Sort data by time and reset indices
    UAdata = UAdata.sort_values(by=['time'])
    LAdata = LAdata.sort_values(by=['time'])
    UBdata = UBdata.sort_values(by=['time'])
    LBdata = LBdata.sort_values(by=['time'])
    UAdata = UAdata.reset_index(drop=True)
    LAdata = LAdata.reset_index(drop=True)
    UBdata = UBdata.reset_index(drop=True)
    LBdata = LBdata.reset_index(drop=True)
    #Rename columns to previous data frames
    UAdata = UAdata.rename(columns={"x":"aXU", "y":"aYU"})
    LAdata = LAdata.rename(columns={"x":"aXL", "y":"aYL"})
    UBdata = UBdata.rename(columns={"x":"bXU", "y":"bYU"})
    LBdata = LBdata.rename(columns={"x":"bXL", "y":"bYL"})
    #Combine separate dataframes to create previous dataframe used
    splitDict = {'aXU':UAdata['aXU'],'aYU':UAdata['aYU'],'aXL':LAdata['aXL'],'aYL':LAdata['aYL'],
                 'bXU':UBdata['bXU'],'bYU':UBdata['bYU'],'bXL':LBdata['bXL'],'bYL':LBdata['bYL'],'time':UAdata['time']}
    posData = pd.DataFrame(data=splitDict)
    #print(posData.head())
    #Calculate initial positioning given Hx, Hy and Theta
    #Add initial positioning
    if Re <= 15.0:
        #SSL
        initDict = {'aXU':[0.0],'aYU':[1.0e-3],'aXL':[0.0],'aYL':[-4.0e-3],
                    'bXU':[Hx*RADIUSLARGE],'bYU':[-Hy*RADIUSLARGE - 1.0e-3],
                    'bXL':[Hx*RADIUSLARGE],'bYL':[-Hy*RADIUSLARGE + 4.0e-3],
                    'time':[0.0]}
    else:
        #LSL
        initDict = {'aXU':[0.0],'aYU':[1.0e-3],'aXL':[0.0],'aYL':[-4.0e-3],
                    'bXU':[Hx*RADIUSLARGE],'bYU':[Hy*RADIUSLARGE - 1.0e-3],
                    'bXL':[Hx*RADIUSLARGE],'bYL':[Hy*RADIUSLARGE + 4.0e-3],
                    'time':[0.0]}
    initData = pd.DataFrame(data=initDict)
    posData = posData.append(initData)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    #print(posData.head())
    #Save only every 20 rows (Every period)
    posData = posData.iloc[::20]
    #Reindex so index corresponds to period number
    posData = posData[posData['time'] <= 40.0].copy()
    posData = posData.reset_index(drop=True)
    
    posData = FixPeriodicBounds(posData)
    
    #Renormalize Position values
    posData['aXU'] = posData['aXU']/RADIUSLARGE
    posData['aYU'] = posData['aYU']/RADIUSLARGE
    posData['aXL'] = posData['aXL']/RADIUSLARGE
    posData['aYL'] = posData['aYL']/RADIUSLARGE
    posData['bXU'] = posData['bXU']/RADIUSLARGE
    posData['bYU'] = posData['bYU']/RADIUSLARGE
    posData['bXL'] = posData['bXL']/RADIUSLARGE
    posData['bYL'] = posData['bYL']/RADIUSLARGE
    #print(posData.head())
    
    #Shift if Re is LSL
    if Re > 15.0:
        #LSL: Shift reference frame
        posData['aXU'] = -1.0*(posData['aXU'] - Hx)
        posData['aYU'] = posData['aYU'] - Hy
        posData['aXL'] = -1.0*(posData['aXL'] - Hx)
        posData['aYL'] = posData['aYL'] - Hy
        posData['bXU'] = -1.0*(posData['bXU'] - Hx)
        posData['bYU'] = posData['bYU'] - Hy
        posData['bXL'] = -1.0*(posData['bXL'] - Hx)
        posData['bYL'] = posData['bYL'] - Hy
    posData['aYU'] = posData['aYU'] +5.0
    posData['aYL'] = posData['aYL'] +5.0
    posData['bYU'] = posData['bYU'] +5.0
    posData['bYL'] = posData['bYL'] +5.0
    posData['aXU'] = posData['aXU'] -1.0
    posData['aXL'] = posData['aXL'] -1.0
    posData['bXU'] = posData['bXU'] -1.0
    posData['bXL'] = posData['bXL'] -1.0
    print(posData.loc[0,'aXU'])
    print(posData.loc[0,'bXU'])
    
    #Calculate Center of mass
    posData['aXCM'] = 0.8*posData['aXU'] + 0.2*posData['aXL']
    posData['aYCM'] = 0.8*posData['aYU'] + 0.2*posData['aYL']
    posData['bXCM'] = 0.8*posData['bXU'] + 0.2*posData['bXL']
    posData['bYCM'] = 0.8*posData['bYU'] + 0.2*posData['bYL']
    #Calculate Distance between swimmers
    posData['Hx'] = posData['bXCM'] - posData['aXCM']
    posData['Hy'] = posData['aYCM'] - posData['bYCM']
    #Calculate Velocity of the swimmers
    posData['avXCM'] = 0.0
    posData['avYCM'] = 0.0
    posData['bvXCM'] = 0.0
    posData['bvYCM'] = 0.0
    posData['alX'] = 0.0
    posData['alY'] = 0.0
    posData['blX'] = 0.0
    posData['blY'] = 0.0
    posData['al'] = 0.0
    posData['bl'] = 0.0
    posData['anX'] = 0.0
    posData['anY'] = 0.0
    posData['bnX'] = 0.0
    posData['bnY'] = 0.0
    for idx in range(len(posData['time'])):
        if idx >= 1 and idx < len(posData['time'])-1:
            posData.loc[idx,'avXCM'] = CentralDifference(posData['aXCM'],idx,PERIOD) 
            posData.loc[idx,'avYCM'] = CentralDifference(posData['aYCM'],idx,PERIOD) 
            posData.loc[idx,'bvXCM'] = CentralDifference(posData['bXCM'],idx,PERIOD) 
            posData.loc[idx,'bvYCM'] = CentralDifference(posData['bYCM'],idx,PERIOD)
        posData.loc[idx,'alX'] = posData.loc[idx,'aXU'] - posData.loc[idx,'aXL']
        posData.loc[idx,'alY'] = posData.loc[idx,'aYU'] - posData.loc[idx,'aYL']
        posData.loc[idx,'blX'] = posData.loc[idx,'bXU'] - posData.loc[idx,'bXL']
        posData.loc[idx,'blY'] = posData.loc[idx,'bYU'] - posData.loc[idx,'bYL']
        posData.loc[idx,'al'] = np.hypot(posData.loc[idx,'alX'],posData.loc[idx,'alY'])
        posData.loc[idx,'bl'] = np.hypot(posData.loc[idx,'blX'],posData.loc[idx,'blY'])
        posData.loc[idx,'anX'] = posData.loc[idx,'alX']/posData.loc[idx,'al']
        posData.loc[idx,'anY'] = posData.loc[idx,'alY']/posData.loc[idx,'al']
        posData.loc[idx,'bnX'] = posData.loc[idx,'blX']/posData.loc[idx,'bl']
        posData.loc[idx,'bnY'] = posData.loc[idx,'blY']/posData.loc[idx,'bl']
    lastIndex = len(posData['time'])-1
    posData.loc[lastIndex,'avXCM'] = posData.loc[lastIndex-1,'avXCM'] 
    posData.loc[lastIndex,'avYCM'] = posData.loc[lastIndex-1,'avYCM'] 
    posData.loc[lastIndex,'bvXCM'] = posData.loc[lastIndex-1,'bvXCM'] 
    posData.loc[lastIndex,'bvYCM'] = posData.loc[lastIndex-1,'bvYCM'] 
        
    #Calculate Angular Velocity of the swimmers
    posData['aTheta'] = CalcLabAngle(posData['anX'],posData['anY'])
    posData['bTheta'] = CalcLabAngle(posData['bnX'],posData['bnY'])
    posData['aw'] = CalcAngularFrequency(posData['aTheta'])
    posData['bw'] = CalcAngularFrequency(posData['bTheta'])
    
    return posData

def PlotSwimmerTrajectories(ax,data,Re,plotNum,color):
    global RADIUSLARGE,RADIUSSMALL
    #Data Values
    lastIndex = len(data['time'])-1
    axU      = data['aXU']
    axL      = data['aXL']
    ayU      = data['aYU']
    ayL      = data['aYL']
    axCM     = data['aXCM']
    ayCM     = data['aYCM']
    bxU      = data['bXU']
    bxL      = data['bXL']
    byU      = data['bYU']
    byL      = data['bYL']
    bxCM     = data['bXCM']
    byCM     = data['bYCM']
    
    #GENERATE FIGURE
    #Phase Space  2D (Hy vs Hx)
    #Swimmer 1
    if plotNum == 1:
        ax.plot(ayCM,axCM,color='purple',zorder=5,alpha=0.5)
        ax.plot(ayU,axU,color='orange',zorder=5,alpha=0.5)
        ax.plot(ayL,axL,color='blue',zorder=5,alpha=0.5)
    elif plotNum == 2:
        #ax.plot(axCM,ayCM,c=cm.viridis(Re/50.0),zorder=5,label='Re = {0}'.format(Re),alpha=0.5)
        ax.plot(ayCM,axCM,color=color,zorder=5,label='Re = {0}'.format(Re),alpha=0.5,lw=2)
        #ax.scatter(ayCM[::20],axCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    elif plotNum == 3:
        #ax.plot(axCM,ayCM,c=cm.winter(Re/15.0),zorder=5,label='Re = {0}'.format(Re),alpha=0.5)
        ax.plot(ayCM,axCM,color=color,zorder=5,label='Re = {0}'.format(Re),alpha=0.5,lw=2)
        ax.scatter(ayCM[::20],axCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    elif plotNum == 4:
        #ax.plot(axCM,ayCM,c=cm.spring((Re-15.0)/50.0),zorder=5,label='Re = {0}'.format(Re),alpha=0.5)
        ax.plot(ayCM,axCM,color=color,zorder=5,label='Re = {0}'.format(Re),alpha=0.5,lw=2)
        ax.scatter(ayCM[::20],axCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    ax.scatter(ayCM[0],axCM[0],c=color,s=60,marker='s',zorder=6,label='_nolegend_')
    ax.scatter(ayCM[lastIndex],axCM[lastIndex],color=color,s=60,zorder=6,label='_nolegend_')
    #Swimmer 2
    if plotNum == 1:
        ax.plot(byCM,bxCM,color='pink',zorder=5,alpha=0.5)
        ax.plot(byU,bxU,color='yellow',zorder=5,alpha=0.5)
        ax.plot(byL,bxL,color='green',zorder=5,alpha=0.5)
    elif plotNum == 2:
        #ax.plot(bxCM,byCM,c=cm.viridis(Re/50.0),zorder=5,label=None,alpha=0.5)
        ax.plot(byCM,bxCM,color=color,zorder=5,label='_nolegend_',alpha=0.5,lw=2)
        #ax.scatter(byCM[::20],bxCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    elif plotNum == 3:
        #ax.plot(bxCM,byCM,c=cm.winter(Re/15.0),zorder=5,label=None,alpha=0.5)
        ax.plot(byCM,bxCM,color=color,zorder=5,label='_nolegend_',alpha=0.5,lw=2)
        ax.scatter(byCM[::20],bxCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    elif plotNum == 4:
        #ax.plot(bxCM,byCM,c=cm.spring((Re-15.0)/50.0),zorder=5,label=None,alpha=0.5)
        ax.plot(byCM,bxCM,color=color,zorder=5,label='_nolegend_',alpha=0.5,lw=2)
        ax.scatter(byCM[::20],bxCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    ax.scatter(byCM[0],bxCM[0],c=color,s=60,marker='s',zorder=6,label='_nolegend_')
    ax.scatter(byCM[lastIndex],bxCM[lastIndex],color=color,s=60,zorder=6,label='_nolegend_')
    
    if plotNum == 1:
        #Add Swimmer locations
        Circle1 = plt.Circle((ayU[lastIndex],axU[lastIndex]),1.0,color='k', clip_on=True)
        ax.add_artist(Circle1)
        Circle2 = plt.Circle((ayL[lastIndex],axL[lastIndex]),0.5,color='k', clip_on=True)
        ax.add_artist(Circle2)
        Circle3 = plt.Circle((byU[lastIndex],bxU[lastIndex]),1.0,color=(0.5,)*3, clip_on=True)
        ax.add_artist(Circle3)
        Circle4 = plt.Circle((byL[lastIndex],bxL[lastIndex]),0.5,color=(0.5,)*3, clip_on=True)
        ax.add_artist(Circle4)
    
    #elif plotNum == 2:
    #    #Add Swimmer locations
    #    Circle1 = plt.Circle((axU[lastIndex],ayU[lastIndex]),1.0,color=cm.viridis(Re/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle1)
    #    Circle2 = plt.Circle((axL[lastIndex],ayL[lastIndex]),0.5,color=cm.viridis(Re/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle2)
    #    Circle3 = plt.Circle((bxU[lastIndex],byU[lastIndex]),1.0,color=cm.viridis(Re/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle3)
    #    Circle4 = plt.Circle((bxL[lastIndex],byL[lastIndex]),0.5,color=cm.viridis(Re/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle4)
    
    #elif plotNum == 3:
    #    #Add Swimmer locations
    #    Circle1 = plt.Circle((axU[lastIndex],ayU[lastIndex]),1.0,color=cm.winter(Re/15.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle1)
    #    Circle2 = plt.Circle((axL[lastIndex],ayL[lastIndex]),0.5,color=cm.winter(Re/15.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle2)
    #    Circle3 = plt.Circle((bxU[lastIndex],byU[lastIndex]),1.0,color=cm.winter(Re/15.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle3)
    #    Circle4 = plt.Circle((bxL[lastIndex],byL[lastIndex]),0.5,color=cm.winter(Re/15.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle4)
    #elif plotNum == 4:
    #    #Add Swimmer locations
    #    Circle1 = plt.Circle((axU[lastIndex],ayU[lastIndex]),1.0,color=cm.spring((Re-15.0)/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle1)
    #    Circle2 = plt.Circle((axL[lastIndex],ayL[lastIndex]),0.5,color=cm.spring((Re-15.0)/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle2)
    #    Circle3 = plt.Circle((bxU[lastIndex],byU[lastIndex]),1.0,color=cm.spring((Re-15.0)/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle3)
    #    Circle4 = plt.Circle((bxL[lastIndex],byL[lastIndex]),0.5,color=cm.spring((Re-15.0)/50.0), clip_on=True,alpha=0.5)
    #    ax.add_artist(Circle4)
    
    
    return ax

def PlotVelocities(ax,time,yval1,yval2,color):
    ax.plot(time/PERIOD,yval1,color=color,zorder=5,label='Re = {0}'.format(Re),alpha=0.5,lw=2)
    ax.scatter(time[::20]/PERIOD,yval1[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    ax.plot(time/PERIOD,yval2,color=color,zorder=5,label='_nolegend_'.format(Re),alpha=0.5,lw=2)
    ax.scatter(time[::20]/PERIOD,yval2[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    return ax

def FindMinValues(data):
    #Xmin
    aXUmin = np.amin(data['aXU'])-1.0
    aXLmin = np.amin(data['aXL'])-1.0
    bXUmin = np.amin(data['bXU'])-1.0
    bXLmin = np.amin(data['bXL'])-1.0
    xmin = min(aXUmin,aXLmin,bXUmin,bXLmin)
    #Ymin
    aYUmin = np.amin(data['aYU'])-1.0
    aYLmin = np.amin(data['aYL'])-1.0
    bYUmin = np.amin(data['bYU'])-1.0
    bYLmin = np.amin(data['bYL'])-1.0
    ymin = min(aYUmin,aYLmin,bYUmin,bYLmin)
    
    return xmin, ymin

def FindMaxValues(data):
    #Xmax
    aXUmax = np.amax(data['aXU'])+1.0
    aXLmax = np.amax(data['aXL'])+1.0
    bXUmax = np.amax(data['bXU'])+1.0
    bXLmax = np.amax(data['bXL'])+1.0
    xmax = max(aXUmax,aXLmax,bXUmax,bXLmax)
    #Ymax
    aYUmax = np.amax(data['aYU'])+1.0
    aYLmax = np.amax(data['aYL'])+1.0
    bYUmax = np.amax(data['bYU'])+1.0
    bYLmax = np.amax(data['bYL'])+1.0
    ymax = max(aYUmax,aYLmax,bYUmax,bYLmax)
    return xmax, ymax

def Set_Tick_Params(ax):
    ax.tick_params(which='major',axis='both',direction='in',length=14,width=1,zorder=10)
    ax.tick_params(which='minor',axis='both',direction='in',length=8,width=0.7)
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_major_locator(MultipleLocator(5))
    # Set tick font size
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(20)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.grid(which='minor',visible=True,zorder=1)
    
def Set_Vel_Tick_Params(ax):
    ax.tick_params(which='major',axis='both',direction='in',length=14,width=1,zorder=10)
    ax.tick_params(which='minor',axis='both',direction='in',length=8,width=0.7)
    ax.xaxis.set_minor_locator(AutoMinorLocator(n=4))
    ax.yaxis.set_minor_locator(AutoMinorLocator(n=4))
    ax.xaxis.set_major_locator(MultipleLocator(100))
    #ax.yaxis.set_major_locator(MultipleLocator(5))
    # Set tick font size
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(20)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)
    return ax



In [184]:
#Get Simulation Data
xminList,xmaxList = [],[]
yminList,ymaxList=[],[]
colorList=['k','purple','tab:blue','tab:green','orange','red','brown']

#Create 4 Figures
#1 for each Re
#2) All Re in one
#3) Only SSL
#4) Only LSL
#Create Figures for Velocity (Linear and Angular)
#5) Linear Velocity (U vs t)
#6) Linear Velocity (V vs t)
#7) Angular Velocity (omega vs t)
#2
fig2, ax2 = plt.subplots(nrows=1,ncols=1,num=2,figsize=(8,8),dpi=250)
#ax2.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax2.set_xlabel(r'y (R)',fontsize=20,**csfont)
ax2.set_ylabel(r'x (R)',fontsize=20,**csfont)
#3
fig3, ax3 = plt.subplots(nrows=1,ncols=1,num=3,figsize=(8,8),dpi=250)
#ax3.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax3.set_xlabel(r'y (R)',fontsize=20,**csfont)
ax3.set_ylabel(r'x (R)',fontsize=20,**csfont)
#4
fig4, ax4 = plt.subplots(nrows=1,ncols=1,num=4,figsize=(8,8),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax4.set_xlabel(r'y (R)',fontsize=20,**csfont)
ax4.set_ylabel(r'x (R)',fontsize=20,**csfont)
#5
fig5, ax5 = plt.subplots(nrows=1,ncols=1,num=5,figsize=(8,6),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax5.set_xlabel(r'time $\tau$',fontsize=24,**csfont)
ax5.set_ylabel(r'$u_x/Rf$',fontsize=24,**csfont)
#6
fig6, ax6 = plt.subplots(nrows=1,ncols=1,num=6,figsize=(8,6),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax6.set_xlabel(r'time $\tau$',fontsize=24,**csfont)
ax6.set_ylabel(r'$u_y/Rf$',fontsize=24,**csfont)
#7
fig7, ax7 = plt.subplots(nrows=1,ncols=1,num=7,figsize=(8,6),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
ax7.set_xlabel(r'time $\tau$',fontsize=24,**csfont)
ax7.set_ylabel(r'$\omega$ (rad/s)',fontsize=24,**csfont)
for idx in range(len(ReList)):
    #1
    Re = ReList[idx]
    color = colorList[idx]
    fig1, ax1 = plt.subplots(nrows=1,ncols=1,num=1,figsize=(8,8),dpi=250)
    ax1.set_xlabel(r'y (R)',fontsize=20,**csfont)
    ax1.set_ylabel(r'x (R)',fontsize=20,**csfont)
    ax1.set_title(r'Re = %.1f'%(Re),fontsize=20,**csfont)
    cwd_POSDATA = cwd_PYTHON+'/../PosData/Re{0}/'.format(Re)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA,Re)
    #Plot #1
    ax1 = PlotSwimmerTrajectories(ax1,simData,Re,1,color)
    xmin, ymin = FindMinValues(simData)
    xmax, ymax = FindMaxValues(simData)
    xminList.append(xmin)
    xmaxList.append(xmax)
    yminList.append(ymin)
    ymaxList.append(ymax)
    ax1.axis([ymin,ymax,xmin,xmax])
    ax1.set_aspect('equal')
    fig1.tight_layout()
    strDir = cwd_PYTHON+"/../Figures/Trajectories/"
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    fig1.savefig(strDir+'Traj_Re{0}_.png'.format(Re))
    fig1.clf()
    #Plot #3
    if Re < 15.0:
        ax3 = PlotSwimmerTrajectories(ax3,simData,Re,3,color)
    #Plot #4
    elif Re >= 15.0:
        ax4 = PlotSwimmerTrajectories(ax4,simData,Re,4,color)
    #Plot #2
    if Re > 15.0:
        ax2 = PlotSwimmerTrajectories(ax2,simData,Re,2,color)
    #Plot Velocities
    if Re < 15.0:
        ax5 = PlotVelocities(ax5,simData['time'],simData['avXCM'],simData['bvXCM'],color)
        ax6 = PlotVelocities(ax6,simData['time'],simData['avYCM'],simData['bvYCM'],color)
        ax7 = PlotVelocities(ax7,simData['time'],simData['aw'],simData['bw'],color)
    print('Re = {0}'.format(Re))
xmin,xmax = np.amin(xminList),np.amax(xmaxList)
ymin,ymax = np.amin(yminList),np.amax(ymaxList)
#2
#ax2.axis([xmin,xmax,ymin,ymax])
ax2.axis([-6,6,-6,6])
ax2.legend(loc='best',fontsize='medium')
Set_Tick_Params(ax2)
ax2.set_aspect('equal')
fig2.tight_layout()
fig2.savefig(strDir+'Traj_LSL_zoom.png')
fig2.clf()
#3
xmin,xmax = np.amin(xminList[:5]),np.amax(xmaxList[:5])
ymin,ymax = np.amin(yminList[:5]),np.amax(ymaxList[:5])
ax3.axis([ymin,ymax,xmin,xmax])
#ax3.legend(loc='best',fontsize='x-small')
Set_Tick_Params(ax3)
ax3.set_aspect('equal')
fig3.tight_layout()
fig3.savefig(strDir+'Traj_SSL_.png')
fig3.clf()
#4
xmin,xmax = np.amin(xminList[5:]),np.amax(xmaxList[5:])
ymin,ymax = np.amin(yminList[5:]),np.amax(ymaxList[5:])
ax4.axis([ymin,ymax,xmin,xmax])
#ax4.legend(loc='best',fontsize='x-small')
Set_Tick_Params(ax4)
ax4.set_aspect('equal')
fig4.tight_layout()
fig4.savefig(strDir+'Traj_LSL_.png')
fig4.clf()
#Save Velocity Figures
ax5.legend(loc='best',fontsize='large')
ax5.set_xlim(0.0,400.0)
Set_Vel_Tick_Params(ax5)
fig5.tight_layout()
fig5.savefig(strDir+'Vel_U_SSL.svg')
fig5.clf()
ax6.legend(loc='best',fontsize='large')
ax6.set_xlim(0.0,400.0)
Set_Vel_Tick_Params(ax6)
fig6.tight_layout()
fig6.savefig(strDir+'Vel_V_SSL.svg')
fig6.clf()
ax7.legend(loc='best',fontsize='large')
ax7.set_xlim(0.0,400.0)
Set_Vel_Tick_Params(ax7)
fig7.tight_layout()
fig7.savefig(strDir+'Vel_Ang_SSL.png')
fig7.clf()
    
plt.close()
print('Plotting is complete!')
                

-1.0
0.0
Re = 0.1
-1.0
0.0
Re = 1.0
-1.0
0.0
Re = 2.0
-1.0
0.0
Re = 7.0
-1.0
0.0
Re = 10.0
0.0
-1.0
Re = 25.0
0.0
-1.0
Re = 50.0
Plotting is complete!


<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

In [259]:
def Set_Traj_Tick_Params(ax):
    ax.tick_params(which='major',axis='both',direction='in',length=14,width=1,zorder=10)
    ax.tick_params(which='minor',axis='both',direction='in',length=8,width=0.7)
    ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(n=5))
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.yaxis.set_major_locator(MultipleLocator(5))
    # Set tick font size
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(20)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.grid(which='both',visible=True,zorder=1)

def ShiftPositions(posData,idx,direction):
    #Shift Data
    if direction == 'SSL':
        posData['aXCM'] = posData['aXCM'] +0.5*(3-idx)
        posData['bXCM'] = posData['bXCM'] +0.5*(3-idx)
    else:
        posData['aXCM'] = posData['aXCM'] +0.5*(5-idx)
        posData['bXCM'] = posData['bXCM'] +0.5*(5-idx)
    
    return posData
    
#Get Simulation Data
xminList,xmaxList = [],[]
yminList,ymaxList=[],[]
colorList=['k','purple','tab:blue','tab:green','orange','red','brown']

#Create 2 Figures
#1) Only SSL
#2) Only LSL
fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(8,8),dpi=250)
ax.set_xlabel(r'x (R)',fontsize=24,**csfont)
ax.set_ylabel(r'y (R)',fontsize=24,**csfont)
fig2, ax2 = plt.subplots(nrows=1,ncols=1,num=2,figsize=(8,8),dpi=250)
ax2.set_xlabel(r'y (R)',fontsize=24,**csfont)
ax2.set_ylabel(r'x (R)',fontsize=24,**csfont)
for idx in range(len(ReList)):
    #1
    Re = ReList[idx]
    color = colorList[idx]
    cwd_POSDATA = cwd_PYTHON+'/../PosData/Re{0}/'.format(Re)
    #Get All sim data and store
    simData = StoreData(cwd_POSDATA,Re)
    xmin, ymin = FindMinValues(simData)
    xmax, ymax = FindMaxValues(simData)
    xminList.append(xmin)
    xmaxList.append(xmax)
    yminList.append(ymin)
    ymaxList.append(ymax)
    #Plot Traj
    simData = ShiftPositions(simData,idx,'SSL')
    ax = PlotSwimmerTrajectories(ax,simData,Re,2,color)
    '''if Re < 15.0:
        #Shift Data so trajectories don't overlap initially
        simData = ShiftPositions(simData,idx,'SSL')
        ax = PlotSwimmerTrajectories(ax,simData,Re,2,color)
    else:
        simData = ShiftPositions(simData,idx,'LSL')
        ax2 = PlotSwimmerTrajectories(ax2,simData,Re,2,color)
    '''
    print('Re = {0}'.format(Re))
strDir = cwd_PYTHON+"/../Figures/Trajectories/"
xmin,xmax = np.amin(xminList[:5]),np.amax(xmaxList[:5])
ymin,ymax = np.amin(yminList[:5]),np.amax(ymaxList[:5])
#1
ax.axis([-7,7,-7,7])
#ax.legend(loc='best',fontsize='medium')
Set_Traj_Tick_Params(ax)
ax.set_aspect('equal')
ax = set_size(6,6,ax)
fig.tight_layout()
fig.savefig(strDir+'Traj_SSL_zoom_shifted.png')
ax.axis([ymin,ymax,xmin,xmax])
fig.savefig(strDir+'Traj_SSL.png')
fig.clf()
#2
#ax2.axis([xmin,xmax,ymin,ymax])
xmin,xmax = np.amin(xminList[5:]),np.amax(xmaxList[5:])
ymin,ymax = np.amin(yminList[5:]),np.amax(ymaxList[5:])
ax2.axis([-6,6,-4.5,4.5])
ax2.legend(loc='best',fontsize='medium')
Set_Traj_Tick_Params(ax2)
ax2.set_aspect('equal')
fig2.tight_layout()
fig2.savefig(strDir+'Traj_LSL_zoom_shifted.png')
ax2.axis([ymin,ymax,xmin,xmax])
fig2.savefig(strDir+'Traj_LSL.png')
fig2.clf()
    
plt.close()
print('Plotting is complete!')
                

-1.0
0.0
Re = 0.1
-1.0
0.0
Re = 1.0
-1.0
0.0
Re = 2.0
-1.0
0.0
Re = 7.0
-1.0
0.0
Re = 10.0
0.0
-1.0
Re = 25.0
0.0
-1.0
Re = 50.0


No handles with labels found to put in legend.


Plotting is complete!


<Figure size 1935.48x1986.75 with 0 Axes>

In [180]:
fig, ax = plt.subplots(nrows=1,ncols=1,num=6,figsize=(8,2.5),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
#ax.set_xlabel(r'y (R)',fontsize=24,**csfont)
#ax.set_ylabel(r'x (R)',fontsize=24,**csfont)

SSL = 0
if SSL:
    #Add Swimmer locations
    Circle1 = plt.Circle((-5.5,0.0),1.0,color=(0.25,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle1)
    Circle2 = plt.Circle((-3.0,0.0),0.5,color=(0.25,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle2)
    Circle3 = plt.Circle((5.5,-1.0),1.0,color=(0.75,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle3)
    Circle4 = plt.Circle((3.0,-1.0),0.5,color=(0.75,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle4)
    #Add Swimmer "springs"
    ax.plot([-5.5,-3.0],
            [0.0,0.0],
            color=(0.25,)*3,linewidth=3,zorder=6)
    ax.plot([5.5,3.0],
            [-1.0,-1.0],
            color=(0.75,)*3,linewidth=3,zorder=6)
else:
    #Add Swimmer locations
    Circle1 = plt.Circle((-4.5,0.0),1.0,color=(0.25,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle1)
    Circle2 = plt.Circle((-7.0,0.0),0.5,color=(0.25,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle2)
    Circle3 = plt.Circle((4.5,-1.0),1.0,color=(0.75,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle3)
    Circle4 = plt.Circle((7.0,-1.0),0.5,color=(0.75,)*3, clip_on=True,zorder=5)
    ax.add_artist(Circle4)
    #Add Swimmer "springs"
    ax.plot([-4.5,-7.0],
            [0.0,0.0],
            color=(0.25,)*3,linewidth=3,zorder=6)
    ax.plot([4.5,7.0],
            [-1.0,-1.0],
            color=(0.75,)*3,linewidth=3,zorder=6)
    
ax.axis([-8,8,-2.5,1.5])
ax.set_aspect('equal')
ax.tick_params(which='major',axis='both',direction='in',length=14,width=1,zorder=10)
ax.tick_params(which='minor',axis='both',direction='in',length=8,width=0.7)
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.yaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_major_locator(MultipleLocator(5))
# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.grid(which='both',visible=True,zorder=1)
strDir = cwd_PYTHON+"/../Figures/Trajectories/"
if SSL:
    fig.savefig(strDir+'SSL_Setup.png')
else:
    fig.savefig(strDir+'LSL_Setup.png')

fig.clf()
plt.close()
print('Plotting Setup is complete')


Plotting Setup is complete


In [252]:
def CheckBounds(delta,val):
    global L
    if delta > L/2.0:
        delta = delta - L
    elif delta < -L/2.0:
        delta = delta + L
    newval = val + delta
    return (delta,newval)

def StoreData(cwd_POSDATA, Re, parHx, parHy, parTheta):
    global RADIUSLARGE, RADIUSSMALL
    #global axAll
    #Reset position data every Sim
    pdData = []
    #Load position data
    pdData1 = pd.read_csv(cwd_POSDATA+'pd.txt',delimiter=' ')
    file = pathlib.Path(cwd_POSDATA+'pd2.txt')
    boolPD = 0
    if file.exists ():
        boolPD = 1
        pdData2 = pd.read_csv(cwd_POSDATA+'/pd2.txt',delimiter=' ')
        pdData = pd.concat([pdData1,pdData2],ignore_index=True)
    else:
        pdData = pdData1.copy() 
    #Load pd3.txt
    file = pathlib.Path(cwd_POSDATA+'pd3.txt')
    if file.exists():
        boolPD = 1
        pdData3 = pd.read_csv(cwd_POSDATA+'/pd3.txt',delimiter=' ')
        pdData = pd.concat([pdData,pdData3],ignore_index=True)
    #Split up individual sphere data by given index
    UAdata = pdData[pdData['idx'] == 6].copy()
    LAdata = pdData[pdData['idx'] == 19].copy()
    UBdata = pdData[pdData['idx'] == 32].copy()
    LBdata = pdData[pdData['idx'] == 45].copy()
    #Sort data by time and reset indices
    UAdata = UAdata.sort_values(by=['time'])
    LAdata = LAdata.sort_values(by=['time'])
    UBdata = UBdata.sort_values(by=['time'])
    LBdata = LBdata.sort_values(by=['time'])
    UAdata = UAdata.reset_index(drop=True)
    LAdata = LAdata.reset_index(drop=True)
    UBdata = UBdata.reset_index(drop=True)
    LBdata = LBdata.reset_index(drop=True)
    #Rename columns to previous data frames
    UAdata = UAdata.rename(columns={"x":"aXU", "y":"aYU"})
    LAdata = LAdata.rename(columns={"x":"aXL", "y":"aYL"})
    UBdata = UBdata.rename(columns={"x":"bXU", "y":"bYU"})
    LBdata = LBdata.rename(columns={"x":"bXL", "y":"bYL"})
    #Combine separate dataframes to create previous dataframe used
    splitDict = {'aXU':UAdata['aXU'],'aYU':UAdata['aYU'],'aXL':LAdata['aXL'],'aYL':LAdata['aYL'],
                 'bXU':UBdata['bXU'],'bYU':UBdata['bYU'],'bXL':LBdata['bXL'],'bYL':LBdata['bYL'],'time':UAdata['time']}
    posData = pd.DataFrame(data=splitDict)
    if boolPD:
        posData = posData.drop_duplicates(subset=["time"])
    #Calculate initial positioning given Hx, Hy and Theta
    #Add initial positioning
    initDict = {'aXU':[0.0],'aYU':[1.0e-3],'aXL':[0.0],'aYL':[-4.0e-3],
                'bXU':[parHx*RADIUSSMALL - 1.0e-3*np.sin(parTheta*np.pi/180.0)],'bYU':[parHy*RADIUSSMALL + 1.0e-3*np.cos(parTheta*np.pi/180.0)],
                'bXL':[parHx*RADIUSSMALL + 4.0e-3*np.sin(parTheta*np.pi/180.0)],'bYL':[parHy*RADIUSSMALL - 4.0e-3*np.cos(parTheta*np.pi/180.0)],
                'time':[0.0]}
    initData = pd.DataFrame(data=initDict)
    posData = posData.append(initData)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    #Save only every 20 rows (Every period)
    posData = posData.iloc[::20]
    #Reindex so index corresponds to period number
    posData = posData.reset_index(drop=True)
    
    #Account for Periodic Conditions
    #Whenever a sphere crosses the boundary
    posData['adXU'] = 0.0
    posData['adYU'] = 0.0
    posData['adXL'] = 0.0
    posData['adYL'] = 0.0
    posData['bdXU'] = 0.0
    posData['bdYU'] = 0.0
    posData['bdXL'] = 0.0
    posData['bdYL'] = 0.0
    posData['aXUn'] = posData['aXU'].copy()
    posData['aYUn'] = posData['aYU'].copy()
    posData['aXLn'] = posData['aXL'].copy()
    posData['aYLn'] = posData['aYL'].copy()
    posData['bXUn'] = posData['bXU'].copy()
    posData['bYUn'] = posData['bYU'].copy()
    posData['bXLn'] = posData['bXL'].copy()
    posData['bYLn'] = posData['bYL'].copy()
    for idx in range(1,len(posData['time'])):
        #Swimmer 1
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'adXU'] = posData.loc[idx,'aXU'] - posData.loc[idx-1,'aXUn']
        posData.loc[idx,'adYU'] = posData.loc[idx,'aYU'] - posData.loc[idx-1,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXU'], posData.loc[idx,'aXUn'] = CheckBounds(posData.loc[idx,'adXU'],posData.loc[idx-1,'aXUn'])
        posData.loc[idx,'adYU'], posData.loc[idx,'aYUn'] = CheckBounds(posData.loc[idx,'adYU'],posData.loc[idx-1,'aYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'adXL'] = posData.loc[idx,'aXL'] - posData.loc[idx,'aXUn']
        posData.loc[idx,'adYL'] = posData.loc[idx,'aYL'] - posData.loc[idx,'aYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'adXL'], posData.loc[idx,'aXLn'] = CheckBounds(posData.loc[idx,'adXL'],posData.loc[idx,'aXUn'])
        posData.loc[idx,'adYL'], posData.loc[idx,'aYLn'] = CheckBounds(posData.loc[idx,'adYL'],posData.loc[idx,'aYUn'])
        #Swimmer 2
        #Ref is previous timestep of large sphere center
        posData.loc[idx,'bdXU'] = posData.loc[idx,'bXU'] - posData.loc[idx-1,'bXUn']
        posData.loc[idx,'bdYU'] = posData.loc[idx,'bYU'] - posData.loc[idx-1,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXU'], posData.loc[idx,'bXUn'] = CheckBounds(posData.loc[idx,'bdXU'],posData.loc[idx-1,'bXUn'])
        posData.loc[idx,'bdYU'], posData.loc[idx,'bYUn'] = CheckBounds(posData.loc[idx,'bdYU'],posData.loc[idx-1,'bYUn'])
        #Ref is other large sphere center
        posData.loc[idx,'bdXL'] = posData.loc[idx,'bXL'] - posData.loc[idx,'bXUn']
        posData.loc[idx,'bdYL'] = posData.loc[idx,'bYL'] - posData.loc[idx,'bYUn']
        #For each delta value, check if it is larger than 0.5*L or smaller than -0.5*L
        posData.loc[idx,'bdXL'], posData.loc[idx,'bXLn'] = CheckBounds(posData.loc[idx,'bdXL'],posData.loc[idx,'bXUn'])
        posData.loc[idx,'bdYL'], posData.loc[idx,'bYLn'] = CheckBounds(posData.loc[idx,'bdYL'],posData.loc[idx,'bYUn'])
        
    
    #Create Swimmer A and Swimmer B dataframes
    #dict_A = {'xU':posData['aXU'],'yU':posData['aYU'],'xL':posData['aXL'],'yL':posData['aYL'],'time':posData['time']}
    #dataA  = pd.DataFrame(data=dict_A)
    #dict_B = {'xU':posData['bXU'],'yU':posData['bYU'],'xL':posData['bXL'],'yL':posData['bYL'],'time':posData['time']}
    #dataB  = pd.DataFrame(data=dict_B)
    dict_A = {'xU':posData['aXUn'],'yU':posData['aYUn'],'xL':posData['aXLn'],'yL':posData['aYLn'],'time':posData['time']}
    dataA  = pd.DataFrame(data=dict_A)
    dict_B = {'xU':posData['bXUn'],'yU':posData['bYUn'],'xL':posData['bXLn'],'yL':posData['bYLn'],'time':posData['time']}
    dataB  = pd.DataFrame(data=dict_B)
    #Create a swimmer class for each
    #print('SwimA')
    swimA = Swimmer(dataA)
    #print('SwimB')
    swimB = Swimmer(dataB)
    angleA = swimA.get_theta()
    angleB = swimB.get_theta()
    #Calculate the three parameters (Hx, Hy, Theta_BW)
    #Hx, Hy, and theta_BW in swimmer A's reference frame
    Hx, Hy, theta_BW = swimB.CalcHxHyTheta(swimA)

    #First, create a dataframe to contain this information
    dict_Param = {'Re':Re,'Hx':Hx,'Hy':Hy,'ThetaBW':theta_BW,'time':posData['time'],
                  'aXU':swimA.xU,'aXL':swimA.xL,'aYU':swimA.yU,'aYL':swimA.yL,'aXCM':swimA.CM[0],'aYCM':swimA.CM[1],
                  'bXU':swimB.xU,'bXL':swimB.xL,'bYU':swimB.yU,'bYL':swimB.yL,'bXCM':swimB.CM[0],'bYCM':swimB.CM[1],}
    posData = pd.DataFrame(data=dict_Param)
    
    #Renormalize Position values
    posData['aXU'] = posData['aXU']/RADIUSLARGE
    posData['aYU'] = posData['aYU']/RADIUSLARGE
    posData['aXL'] = posData['aXL']/RADIUSLARGE
    posData['aYL'] = posData['aYL']/RADIUSLARGE
    posData['bXU'] = posData['bXU']/RADIUSLARGE
    posData['bYU'] = posData['bYU']/RADIUSLARGE
    posData['bXL'] = posData['bXL']/RADIUSLARGE
    posData['bYL'] = posData['bYL']/RADIUSLARGE
    posData['aXCM'] = posData['aXCM']/RADIUSLARGE
    posData['aYCM'] = posData['aYCM']/RADIUSLARGE
    posData['bXCM'] = posData['bXCM']/RADIUSLARGE
    posData['bYCM'] = posData['bYCM']/RADIUSLARGE
    #Shift Data
    posData['aXU'] = posData['aXU'] -2.125
    posData['aXL'] = posData['aXL'] -2.125
    posData['bXU'] = posData['bXU'] -2.125
    posData['bXL'] = posData['bXL'] -2.125
    posData['aXCM'] = posData['aXCM'] -2.125
    posData['bXCM'] = posData['bXCM'] -2.125
    
    
    
    return posData

def Rotate(xy, theta):
    # https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    #First Rotate based on Theta
    #Allocate Arrays
    rotationMatrix = np.zeros((2,2))
    #Calculate rotation matrix
    rotationMatrix[0,0] = np.cos(theta)
    rotationMatrix[0,1] = -1.0*np.sin(theta)
    rotationMatrix[1,0] = np.sin(theta)
    rotationMatrix[1,1] = np.cos(theta) 
    return rotationMatrix.dot(xy)

def Translate(xy, offset):
    return xy + offset

class Swimmer:
    def __init__(self,data):
        self.data  = data
        self.xU, self.yU = data['xU'], data['yU']
        self.xL, self.yL = data['xL'], data['yL']
        self.time = data['time']
        self.CM = np.zeros((2,len(self.time)))
        self.theta = np.zeros(len(self.time))
        self.normY = np.zeros((2,len(self.time)))
        self.normX = np.zeros((2,len(self.time)))
        self.Hx = np.zeros(len(self.time))
        self.Hy = np.zeros(len(self.time))
        self.rot_norm = np.zeros((2,len(self.time)))
        self.theta_BW = np.zeros(len(self.time))
        self.normLSS = np.zeros((2,len(self.time)))
        self.CalcCM()
        self.CalcLabAngle()
    
    def get_theta(self):
        return self.theta
    
    def get_norms(self):
        return (self.normX, self.normY)
        
    def CalcCM(self):
        self.xCM = 0.8*self.xU + 0.2*self.xL
        self.yCM = 0.8*self.yU + 0.2*self.yL
        self.CM[0] = self.xCM
        self.CM[1] = self.yCM
        #print('CM = ',self.CM[:,0:10])
        
    def CalcLabAngle(self):
        #Find swimming axis (normal y-axis)
        self.labX   = self.xU - self.xL
        self.labY   = self.yU - self.yL
        self.length = np.hypot(self.labX,self.labY)
        self.normY[0] = self.labX/self.length
        self.normY[1] = self.labY/self.length
        #self.normY  = np.array([self.labX/self.length,self.labY,self.length])
        #2) Calculate Theta
        for idx in range(len(self.time)):
            if(self.normY[1,idx] >= 0.0):
                self.theta[idx] = np.arccos(self.normY[0,idx])
                #Theta_a[idx] = np.arccos(norm_aX[idx])
            else:
                self.theta[idx] = -1.0*np.arccos(self.normY[0,idx])+2.0*np.pi
                #Theta_a[idx] = -1.0*np.arccos(norm_aX[idx])+2.0*np.pi
        #Calculate perpendicular axis (swimmer's x-axis)
        self.normX[0] = np.cos(self.theta - 0.5*np.pi)
        self.normX[1] = np.sin(self.theta - 0.5*np.pi)
        
    def CalcHxHyTheta(self,swimmer):
        #Here, we will calculate three parameters
        #1) Hx: In the swimmer A's reference frame
        #2) Hy: In the swimmer A's reference frame
        #3) Theta_BW: Angle between swimmer A and swimmerB (w/ respect to swimmer A y-axis)
        #But first, we must calculate in lab frame for some
        
        #1) Hx: In swimmer A's reference frame
        #2) Hy: In swimmer A's reference frame
        self.Projection(swimmer.normX,swimmer.normY,swimmer.CM)
        
        #3) Rotate A and B ccw by 2pi - Theta_a
        self.rotateAngle = 2.0*np.pi - swimmer.theta
        for idx in range(len(self.length)):
            self.rot_norm[:,idx] = Rotate(self.normY[:,idx],self.rotateAngle[idx])
            swimmer.rot_norm[:,idx] = Rotate(swimmer.normY[:,idx],self.rotateAngle[idx])
            if(self.rot_norm[1,idx] >= 0.0):
                self.theta_BW[idx] = np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))%(2.0*np.pi)
            else:
                self.theta_BW[idx] = (-1.0*np.arccos(swimmer.rot_norm[:,idx].dot(self.rot_norm[:,idx]))+2.0*np.pi)%(2.0*np.pi)
        
        return (self.Hx/RADIUSLARGE, self.Hy/RADIUSLARGE, self.theta_BW)
    
    def Projection(self,normX,normY,CM):
        #Here we find the distance component for swimmer A's axes
        #print('normY = ',normY)
        #print('CM = ',self.CM)
        for idx in range(len(self.time)):
            self.Hx[idx] = np.dot(normX[:,idx],(self.CM[:,idx]-CM[:,idx]))
            self.Hy[idx] = np.dot(normY[:,idx],(self.CM[:,idx]-CM[:,idx]))


In [253]:
#Get Side-by-side trajectories for Re=2, 7, 10, and 25

HxVal = 8.5
HyVal = 0
Theta = 0.0
ReList = [2,7,10,25]

dict_Param = {'Re':[],'Hx':[],'Hy':[],'ThetaBW':[],'time':[],
              'aXU':[],'aXL':[],'aYU':[],'aYL':[],'aXCM':[],'aYCM':[],
              'bXU':[],'bXL':[],'bYU':[],'bYL':[],'bXCM':[],'bYCM':[],}
sideData = pd.DataFrame(data=dict_Param)

for Re in ReList:
    cwd_POSDATA = cwd_PYTHON+'/../../PairDynamics/PosData/Re{0}/Theta{1}/Hx{2}/Hy{3}/'.format(Re,Theta,HxVal,HyVal)
    simData = StoreData(cwd_POSDATA,Re,HxVal,HyVal,Theta)
    simData = simData[simData['time'] <= 20.0].copy()
    sideData = pd.concat([sideData,simData],ignore_index=True)
    print('Re = {0} is complete!'.format(Re))
sideData = sideData.sort_values(by=['Re','time']).reset_index(drop=True)



Re = 2 is complete!
Re = 7 is complete!
Re = 10 is complete!
Re = 25 is complete!


In [255]:
def PlotSideSwimmerTrajectories(ax,data,Re,plotNum,color):
    global RADIUSLARGE,RADIUSSMALL
    #Data Values
    lastIndex = len(data['time'])-1
    axU      = data['aXU']
    axL      = data['aXL']
    ayU      = data['aYU']
    ayL      = data['aYL']
    axCM     = data['aXCM']
    ayCM     = data['aYCM']
    bxU      = data['bXU']
    bxL      = data['bXL']
    byU      = data['bYU']
    byL      = data['bYL']
    bxCM     = data['bXCM']
    byCM     = data['bYCM']
    
    #GENERATE FIGURE
    #Phase Space  2D (Hy vs Hx)
    #Swimmer 1
    if plotNum == 2:
        #ax.plot(axCM,ayCM,c=cm.viridis(Re/50.0),zorder=5,label='Re = {0}'.format(Re),alpha=0.5)
        ax.plot(axCM,ayCM,color=color,zorder=5,label='Re = {0}'.format(Re),alpha=0.5,lw=2)
        #ax.scatter(axCM[::20],ayCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    ax.scatter(axCM[0],ayCM[0],c='k',s=60,marker='s',zorder=6,label='_nolegend_')
    ax.scatter(axCM[lastIndex],ayCM[lastIndex],color=color,s=60,zorder=6,label='_nolegend_')
    #Swimmer 2
    if plotNum == 2:
        #ax.plot(bxCM,byCM,c=cm.viridis(Re/50.0),zorder=5,label=None,alpha=0.5)
        ax.plot(bxCM,byCM,color=color,zorder=5,label='_nolegend_',alpha=0.5,lw=2)
        #ax.scatter(bxCM[::20],byCM[::20],color=color,zorder=5,label='_nolegend_',alpha=0.5,s=36)
    ax.scatter(bxCM[0],byCM[0],c='k',s=60,marker='s',zorder=6,label='_nolegend_')
    ax.scatter(bxCM[lastIndex],byCM[lastIndex],color=color,s=60,zorder=6,label='_nolegend_')
    
    
    return ax

#Get Simulation Data
xminList,xmaxList = [],[]
yminList,ymaxList=[],[]
colorList=['tab:blue','tab:green','orange','red']

#Create 2 Figures
#1) Only SSL
#2) Only LSL
fig, ax = plt.subplots(nrows=1,ncols=1,num=1,figsize=(8,8),dpi=250)
ax.set_xlabel(r'x (R)',fontsize=24,**csfont)
ax.set_ylabel(r'y (R)',fontsize=24,**csfont)
for idx in range(len(ReList)):
    #1
    Re = ReList[idx]
    color = colorList[idx]
    simData = sideData[sideData['Re'] == Re].copy()
    simData = simData.reset_index(drop=True)
    xmin, ymin = FindMinValues(simData)
    xmax, ymax = FindMaxValues(simData)
    xminList.append(xmin)
    xmaxList.append(xmax)
    yminList.append(ymin)
    ymaxList.append(ymax)
    #Plot Traj
    ax = PlotSideSwimmerTrajectories(ax,simData,Re,2,color)
    print('Re = {0}'.format(Re))
strDir = cwd_PYTHON+"/../Figures/Trajectories/"
xmin,xmax = np.amin(xminList),np.amax(xmaxList)
ymin,ymax = np.amin(yminList),np.amax(ymaxList)
#1
ax.axis([-9,9,-16.5,1.5])
#ax.legend(loc='best',fontsize='medium')
Set_Traj_Tick_Params(ax)
ax.set_aspect('equal')
ax = set_size(6,6,ax)
fig.tight_layout()
fig.savefig(strDir+'Traj_sbs_zoom.png')
ax.axis([xmin,xmax,ymin,ymax])
fig.savefig(strDir+'Traj_sbs.png')
fig.clf()
    
plt.close()
print('Plotting is complete!')

Re = 2
Re = 7
Re = 10
Re = 25
Plotting is complete!


In [217]:
fig, ax = plt.subplots(nrows=1,ncols=1,num=6,figsize=(5,5),dpi=250)
#ax4.set_title(r'time =%.1f: $\theta$ = %s: $H_y$ = %.1f vs. $H_x$ = %.2f'%(time,Theta,Hy,Hx),fontsize=15,**csfont)
#ax.set_xlabel(r'y (R)',fontsize=24,**csfont)
#ax.set_ylabel(r'x (R)',fontsize=24,**csfont)

#Add Swimmer locations
Circle1 = plt.Circle((-2.125,0.5),1.0,color=(0.25,)*3, clip_on=True,zorder=5)
ax.add_artist(Circle1)
Circle2 = plt.Circle((-2.125,-2.0),0.5,color=(0.25,)*3, clip_on=True,zorder=5)
ax.add_artist(Circle2)
Circle3 = plt.Circle((2.125,0.5),1.0,color=(0.75,)*3, clip_on=True,zorder=5)
ax.add_artist(Circle3)
Circle4 = plt.Circle((2.125,-2.0),0.5,color=(0.75,)*3, clip_on=True,zorder=5)
ax.add_artist(Circle4)
#Add Swimmer "springs"
ax.plot([-2.125,-2.125],
        [0.5,-2.0],
        color=(0.25,)*3,linewidth=3,zorder=6)
ax.plot([2.125,2.125],
        [0.5,-2.0],
        color=(0.75,)*3,linewidth=3,zorder=6)
    
ax.axis([-4,4,-4,4])
ax.set_aspect('equal')
ax.tick_params(which='major',axis='both',direction='in',length=14,width=1,zorder=10)
ax.tick_params(which='minor',axis='both',direction='in',length=8,width=0.7)
ax.xaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.yaxis.set_minor_locator(AutoMinorLocator(n=5))
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_major_locator(MultipleLocator(5))
# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(20)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
ax.grid(which='both',visible=True,zorder=1)
strDir = cwd_PYTHON+"/../Figures/Trajectories/"
fig.savefig(strDir+'SBS_Setup.png')

fig.clf()
plt.close()
print('Plotting Setup is complete')


Plotting Setup is complete
